<a href="https://colab.research.google.com/github/Jose897/7506-OrganizacionDeDatos/blob/main/Clase_1_Yo_Spark_BigMac.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instalamos e importamos librerías

In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt update
!apt install openjdk-8-jdk-headless -qq
#!apt install default-jre
#!apt install default-jdk
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:15 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

# Autenticamos con Google Drive

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Bajamos archivo con los datos del índice BigMac

In [ ]:
downloaded = drive.CreateFile({'id':"1h9Lrqmc4uAkXeC0DJAg3f4VlJLut5C7S"})
downloaded.GetContentFile('bigmac.csv') 

# Creamos el Spark Context

In [ ]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

## Leemos CSV

In [ ]:
sqlContext = SQLContext(sc)
df = sqlContext.read.csv('bigmac.csv', header=True, inferSchema=True)
rdd = df.rdd

## Cantidad de registros

In [ ]:
rdd.count()

1730

## Qué datos tiene cada registro

In [ ]:
rdd.take(5)

[Row(name='Argentina', iso_a3='ARG', currency_code='ARS', local_price=2.5, dollar_ex=1.0, GDP_dollar=None, date='2000-04-01'),
 Row(name='Australia', iso_a3='AUS', currency_code='AUD', local_price=2.59, dollar_ex=1.68, GDP_dollar=None, date='2000-04-01'),
 Row(name='Brazil', iso_a3='BRA', currency_code='BRL', local_price=2.95, dollar_ex=1.79, GDP_dollar=None, date='2000-04-01'),
 Row(name='Britain', iso_a3='GBR', currency_code='GBP', local_price=1.9, dollar_ex=0.632911392, GDP_dollar=None, date='2000-04-01'),
 Row(name='Canada', iso_a3='CAN', currency_code='CAD', local_price=2.85, dollar_ex=1.47, GDP_dollar=None, date='2000-04-01')]

## Qué país y cuando tuvo el mayor valor en USD

In [ ]:
rdd = rdd.filter(lambda x: x.dollar_ex != 0).cache()

In [ ]:
rdd.reduce(lambda x,y: x if x.local_price / x.dollar_ex > y.local_price / y.dollar_ex else y)

Row(name='Venezuela', iso_a3='VEN', currency_code='VEF', local_price=39.0, dollar_ex=4.29465, GDP_dollar=None, date='2013-01-01')

In [ ]:
rdd.filter(lambda x: x.iso_a3 != 'VEN').reduce(lambda x,y: x if x.local_price / x.dollar_ex > y.local_price / y.dollar_ex else y)

Row(name='Norway', iso_a3='NOR', currency_code='NOK', local_price=45.0, dollar_ex=5.41405, GDP_dollar=84443.634, date='2011-07-01')

## Y el menor?

In [ ]:
rdd.reduce(lambda x,y: x if x.local_price / x.dollar_ex < y.local_price / y.dollar_ex else y)

Row(name='Saudi Arabia', iso_a3='SAU', currency_code='SAR', local_price=2.4, dollar_ex=3.7502, GDP_dollar=None, date='2004-05-01')

## Precios de Argentina

In [ ]:
rdd.filter(lambda x: x.iso_a3 == 'ARG').collect()

[Row(name='Argentina', iso_a3='ARG', currency_code='ARS', local_price=2.5, dollar_ex=1.0, GDP_dollar=None, date='2000-04-01'),
 Row(name='Argentina', iso_a3='ARG', currency_code='ARS', local_price=2.5, dollar_ex=1.0, GDP_dollar=None, date='2001-04-01'),
 Row(name='Argentina', iso_a3='ARG', currency_code='ARS', local_price=2.5, dollar_ex=3.13, GDP_dollar=None, date='2002-04-01'),
 Row(name='Argentina', iso_a3='ARG', currency_code='ARS', local_price=4.1, dollar_ex=2.88, GDP_dollar=None, date='2003-04-01'),
 Row(name='Argentina', iso_a3='ARG', currency_code='ARS', local_price=4.36, dollar_ex=2.95, GDP_dollar=None, date='2004-05-01'),
 Row(name='Argentina', iso_a3='ARG', currency_code='ARS', local_price=4.75, dollar_ex=2.897, GDP_dollar=None, date='2005-06-01'),
 Row(name='Argentina', iso_a3='ARG', currency_code='ARS', local_price=4.75, dollar_ex=3.0638, GDP_dollar=None, date='2006-01-01'),
 Row(name='Argentina', iso_a3='ARG', currency_code='ARS', local_price=7.0, dollar_ex=3.0565, GDP_dol

### En USD

In [ ]:
arg = rdd.filter(lambda x: x.iso_a3 == 'ARG')

In [ ]:
arg_usd = arg.map(lambda x: (x.date, x.local_price/x.dollar_ex)).cache()

In [ ]:
arg_usd.collect()

[('2000-04-01', 2.5),
 ('2001-04-01', 2.5),
 ('2002-04-01', 0.7987220447284346),
 ('2003-04-01', 1.423611111111111),
 ('2004-05-01', 1.4779661016949153),
 ('2005-06-01', 1.6396272005522956),
 ('2006-01-01', 1.550362295188981),
 ('2006-05-01', 2.290201210534925),
 ('2007-01-01', 2.670983407527317),
 ('2007-06-01', 2.668607472100922),
 ('2008-06-01', 3.642987249544627),
 ('2009-07-01', 3.0173827484421123),
 ('2010-01-01', 1.842832697117283),
 ('2010-07-01', 3.558945027009851),
 ('2011-07-01', 4.839685420447671),
 ('2012-01-01', 4.636606004404776),
 ('2012-07-01', 4.160963591568573),
 ('2013-01-01', 3.8179443383904355),
 ('2013-07-01', 3.8799076212471135),
 ('2014-01-01', 3.0341340075853354),
 ('2014-07-01', 2.5707727620504977),
 ('2015-01-01', 3.2520325203252036),
 ('2015-07-01', 3.0651340996168583),
 ('2016-01-01', 2.3897025544472004),
 ('2016-07-01', 3.3478406427854033),
 ('2017-01-01', 3.4683903515686585),
 ('2017-07-01', 4.1255341093266535),
 ('2018-01-01', 3.9603960396039604),
 ('20

### Mayor

In [ ]:
arg_usd.takeOrdered(3, lambda x: -x[1])

[('2011-07-01', 4.839685420447671),
 ('2012-01-01', 4.636606004404776),
 ('2012-07-01', 4.160963591568573)]

### Menor

In [ ]:
arg_usd.takeOrdered(3, lambda x: x[1])

[('2002-04-01', 0.7987220447284346),
 ('2003-04-01', 1.423611111111111),
 ('2004-05-01', 1.4779661016949153)]

### 5 mayores

### 5 menores

### Valor promedio

In [ ]:
arg_usd.map(lambda x: x[1]).reduce(lambda x,y: x+y) / arg_usd.count()

2.9356961998694073

## Valores en USA

In [ ]:
rdd.filter(lambda x: x.iso_a3 == 'USA').map(lambda x: (x.date, x.local_price)).collect()

[('2000-04-01', 2.51),
 ('2001-04-01', 2.54),
 ('2002-04-01', 2.49),
 ('2003-04-01', 2.71),
 ('2004-05-01', 2.9),
 ('2005-06-01', 3.06),
 ('2006-01-01', 3.15),
 ('2006-05-01', 3.1),
 ('2007-01-01', 3.22),
 ('2007-06-01', 3.41),
 ('2008-06-01', 3.57),
 ('2009-07-01', 3.57),
 ('2010-01-01', 3.58),
 ('2010-07-01', 3.733333333),
 ('2011-07-01', 4.065),
 ('2012-01-01', 4.19722),
 ('2012-07-01', 4.3275),
 ('2013-01-01', 4.367395833),
 ('2013-07-01', 4.556666667),
 ('2014-01-01', 4.624166667),
 ('2014-07-01', 4.795),
 ('2015-01-01', 4.79),
 ('2015-07-01', 4.79),
 ('2016-01-01', 4.93),
 ('2016-07-01', 5.04),
 ('2017-01-01', 5.06),
 ('2017-07-01', 5.3),
 ('2018-01-01', 5.28),
 ('2018-07-01', 5.51),
 ('2019-01-01', 5.58),
 ('2019-07-09', 5.74),
 ('2020-01-14', 5.67),
 ('2020-07-01', 5.71),
 ('2021-01-01', 5.66)]

## 10 más caros

In [ ]:
rdd.map(lambda x: (x.name, (x.local_price / x.dollar_ex, 1))).reduceByKey(lambda x,y: (x[0]+y[0],x[1]+y[1]))\
   .map(lambda x: (x[0],x[1][0]/x[1][1]))\
   .takeOrdered(10, lambda x:-x[1])

[('Norway', 6.380225767510909),
 ('Switzerland', 6.1111973283220085),
 ('Sweden', 5.21455311225687),
 ('Finland', 5.1629535999708125),
 ('Italy', 4.7913481249763565),
 ('France', 4.76472499997649),
 ('Belgium', 4.749227374984401),
 ('Denmark', 4.714949725134277),
 ('Ireland', 4.640071099992816),
 ('Germany', 4.599525249984549)]

In [ ]:
rdd.map(lambda x: (x.name, (x.local_price / x.dollar_ex, 1))).reduceByKey(lambda x,y: (x[0]+y[0],x[1]+y[1]))\
   .takeOrdered(10, lambda x:-x[1][0]/x[1][1])

[('Norway', (191.40677302532728, 30)),
 ('Switzerland', (207.78070916294828, 34)),
 ('Sweden', (177.29480581673357, 34)),
 ('Finland', (103.25907199941625, 20)),
 ('Italy', (95.82696249952713, 20)),
 ('France', (95.2944999995298, 20)),
 ('Belgium', (94.98454749968802, 20)),
 ('Denmark', (160.3082906545654, 34)),
 ('Ireland', (92.80142199985632, 20)),
 ('Germany', (91.99050499969098, 20))]

## 10 más baratos